In [1]:
import polars as pl
import numpy as np
import time
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [ ]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [12]:
def startTrain(X, y, penalty, C, solver):
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # 🔹 Normalização MinMaxScaler
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Aplicando o undersampling no conjunto de treino
        rus = RandomUnderSampler()
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)

        start_training = time.time()

        # Inicializa o modelo de Regressão Logística
        lr = LogisticRegression(penalty=penalty, C=C, solver=solver)
        
        # Treina o modelo
        lr.fit(X_train_resampled, y_train_resampled)
        end_training = time.time()

        # Predição
        y_pred = lr.predict(X_test_scaled)
        evaluation_duration = time.time() - end_training
        training_duration = end_training - start_training
        
        confusion = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        specificity = tn / (tn + fp)
        f1 = f1_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        #print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["LR"] + mean_results.tolist())

In [13]:
#Melhores Hiperparâmetros encontrados: {'penalty': 'l2', 'C': 1, 'solver': 'newton-cg'}

In [14]:
for i in range(1,11):
    startTrain(X=X, y=y, penalty='l2', C=1, solver='newton-cg')
    print(i)

1
2
3
4
5
6
7
8
9
10


In [15]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""LR""",0.981201,0.973271,0.992612,0.984943,0.961599,0.988762,0.038401,17119.599609,683.599976,1404.0,91839.398438,0.160195,0.002337
"""LR""",0.981197,0.973272,0.992611,0.984939,0.961606,0.98876,0.038394,17119.599609,683.599976,1404.400024,91839.0,0.147171,0.002217
"""LR""",0.981195,0.973256,0.992605,0.984942,0.961569,0.988759,0.038431,17119.0,684.200012,1404.0,91839.398438,0.151487,0.002001
"""LR""",0.981204,0.973254,0.992601,0.984958,0.961551,0.988764,0.038449,17118.599609,684.599976,1402.599976,91840.796875,0.136816,0.002001
"""LR""",0.981211,0.973295,0.992618,0.984949,0.96164,0.988769,0.03836,17120.199219,683.0,1403.400024,91840.0,0.145472,0.002906
"""LR""",0.981194,0.973252,0.992603,0.984943,0.961562,0.988758,0.038438,17118.800781,684.400024,1404.0,91839.398438,0.137118,0.001417
"""LR""",0.981199,0.97327,0.992609,0.984943,0.961597,0.988761,0.038403,17119.400391,683.799988,1404.0,91839.398438,0.141304,0.002046
"""LR""",0.981211,0.973306,0.992624,0.984943,0.96167,0.988768,0.03833,17120.800781,682.400024,1404.0,91839.398438,0.144015,0.002213
"""LR""",0.981204,0.973292,0.992618,0.984941,0.961644,0.988764,0.038356,17120.199219,683.0,1404.199951,91839.203125,0.147293,0.001801


In [ ]:
metrics_df.write_csv("metrics_results/balanced_lr_metrics_output.csv", separator=';')